In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from topostats.filters import Filters
from topostats.io import LoadScans
from topostats.plottingfuncs import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()

In [ ]:
file_path = Path("/Users/sylvi/Downloads/20230818_p4cat_SC_mgni_3ng.0_00022.spm")
assert file_path.exists()

loadscans = LoadScans(img_paths=[file_path], channel="Height")
loadscans.get_data()

data = loadscans.img_dict
print(data.keys())
image_data = data[list(data.keys())[0]]
print(image_data.keys())
image_raw = image_data["image_original"]
p2nm = image_data["pixel_to_nm_scaling"]
print(f"{p2nm=}")
print(f"Shape: {image_raw.shape}")

plt.imshow(image_raw, cmap=cmap)
plt.show()

# Flatten the image
filters = Filters(
    image=image_raw,
    filename="test",
    pixel_to_nm_scaling=p2nm,
    row_alignment_quantile=0.5,
    threshold_method="std_dev",
    threshold_std_dev={"above": 1.0, "below": 10.0},
    gaussian_size=1.01,
    gaussian_mode="nearest",
    remove_scars={
        "run": False,
    },
)
filters.filter_image()
flattened_image = filters.images["gaussian_filtered"]
plt.imshow(flattened_image, cmap=cmap)
plt.show()

# Get the size of the image in bytes
size_bytes = flattened_image.nbytes
print(f"{size_bytes=}")
# Convert to np.float16 to see if the size is smaller
flattened_image_16 = flattened_image.astype(np.float16)
size_bytes_16 = flattened_image_16.nbytes
print(f"{size_bytes_16=}")

print(f"ratio: {size_bytes_16/size_bytes}")

# print a number from the 16 bit image to see what the precision is
print(f"16 bit image: {flattened_image_16[0,0]}")
print(f"32 bit image: {flattened_image[0,0]}")

# Save the image as a npy array
np.save("./tests/resources/example_catenane.npy", flattened_image_16)

In [ ]:
# Find the grains